In [1]:
from typing import List, Any, Tuple, Dict, Set
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_utilization,
)
from datasets import Dataset, Features, Sequence, Value
from langchain_openai import AzureChatOpenAI

# Assume that the previous notebooks have been run to load environment variables, imports, embeddings, chunked texts, vector stores, RAG chains, and conversational RAG chains
%run "conversational_ragchain.ipynb"

# Initialize the AzureChatOpenAI model
llm = AzureChatOpenAI(
    openai_api_version=os.getenv('AZURE_API_VERSION'),
    azure_deployment=os.getenv('AZURE_DEPLOYMENT'),
    api_key=os.getenv('AZURE_API_KEY'),
    azure_endpoint=os.getenv('AZURE_ENDPOINT')
)

def evaluate_answer(answer: str, question: str, context: List[Any], ground_truth: str) -> Dict[str, float]:
    # Format the context as a list of strings for 'contexts'
    context_str = format_context(context)
    context_list = context_str.split('\n\n')

    # Create a dictionary with lists for each column
    data = {
        "question": [question],
        "context": [context_str],
        "answer": [answer],
        "contexts": [context_list],  # 'contexts' as a list of strings
        "ground_truth": [ground_truth]  # 'ground_truth' column
    }

    # Define the feature types
    features = Features({
        "question": Value("string"),
        "context": Value("string"),
        "answer": Value("string"),
        "contexts": Sequence(Value("string")),
        "ground_truth": Value("string")
    })

    # Convert to HuggingFace dataset
    dataset = Dataset.from_dict(data, features=features)

    # Evaluate using RAGAS
    scores = evaluate(
        dataset,
        metrics=[context_utilization, faithfulness, answer_relevancy, context_recall],
        column_map={"question": "question", "context": "context", "answer": "answer", "contexts": "contexts", "ground_truth": "ground_truth"},
        llm=llm,  # Pass the AzureChatOpenAI model
        embeddings=embedding_function  # Pass the SentenceTransformerEmbeddings
    )
    return scores

def get_aggregated_answer_and_sources(input_question: str, session_id="abc123") -> Tuple[str, Dict[str, str], List[Any], Set[Tuple[str, str]]]:
    def is_invalid_answer(answer: str) -> bool:
        return "I don't" in answer or "I'm sorry" in answer

    def get_response_and_sources(response: Dict[str, Any], all_texts: List[Any]) -> Tuple[str, List[Any], Set[Tuple[str, str]]]:
        answer = response["answer"]
        context = response["context"]
        used_chunks = [doc for doc in all_texts if any(chunk.page_content in doc.page_content for chunk in context)]
        sources = {(chunk.metadata['file'], chunk.metadata.get('page', 'Unknown')) for chunk in used_chunks}
        return answer, context, sources

    all_responses = {}
    all_contexts = []
    all_sources = set()

    for key in ['chroma', 'faiss', 'pinecone']:
        response = conversational_rag_chains[key].invoke(
            {"input": input_question},
            config={"configurable": {"session_id": session_id}},
        )
        if not is_invalid_answer(response["answer"]):
            answer, context, sources = get_response_and_sources(response, all_texts[key])
            all_responses[key] = answer
            all_contexts.extend(context)
            all_sources.update(sources)

    if not all_responses:
        return "I don't know the answer.", {}, [], set()

    # Combine all the valid responses into one, filtering out duplicates
    sentences = []
    for answer in all_responses.values():
        sentences.extend(answer.split(". "))

    # Embed the sentences to filter out duplicates
    sentence_embeddings = embedding_function.embed_documents(sentences)
    unique_sentences = []
    unique_embeddings = []

    for i, embedding in enumerate(sentence_embeddings):
        if not unique_embeddings:  # First sentence
            unique_sentences.append(sentences[i])
            unique_embeddings.append(embedding)
        else:
            similarities = cosine_similarity([embedding], unique_embeddings)
            if all(sim < 0.8 for sim in similarities[0]):  # Adjust the threshold as needed
                unique_sentences.append(sentences[i])
                unique_embeddings.append(embedding)

    aggregated_answer = ". ".join(unique_sentences)

    return aggregated_answer, all_responses, all_contexts, all_sources

def format_context(context: List[Any]) -> str:
    formatted_context = "\n\n".join([
        f"File: {doc.metadata['file']}, Page {doc.metadata.get('page', 'Unknown')}: {doc.page_content}"
        for doc in context
    ])
    return formatted_context



AZURE_API_KEY: d5c1ae66fe5a41fd82f61e0b9fe88470
AZURE_API_VERSION: 2024-02-15-preview
AZURE_DEPLOYMENT: gpt-4t
AZURE_ENDPOINT: https://itp-azopenai-ne.openai.azure.com/


C:\Users\vlad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\vlad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
# Usage Example
question = "What are the primary tasks performed by broadcast technicians during radio and television broadcasts?"
aggregated_answer, all_responses, context, sources = get_aggregated_answer_and_sources(question)

# Format the output in a readable way
formatted_context = format_context(context)

output = f"""
Final Aggregated Answer: {aggregated_answer}

Intermediate Answers:
Chroma: {all_responses.get('chroma', 'No valid answer found')}
FAISS: {all_responses.get('faiss', 'No valid answer found')}
Pinecone: {all_responses.get('pinecone', 'No valid answer found')}

Context used for answer:
{formatted_context}
"""

print(output)

# Assuming ground_truth is known for this example
ground_truth = "Primary tasks performed by broadcast technicians during radio and television broadcasts include operating and maintaining equipment, monitoring signal strength, and ensuring quality of audio and video transmissions."

# Evaluate the final aggregated answer
final_scores = evaluate_answer(aggregated_answer, question, context, ground_truth)
print("\nFinal Aggregated Answer Evaluation Scores:", final_scores)

# Evaluate individual retriever answers
for key, answer in all_responses.items():
    retriever_scores = evaluate_answer(answer, question, context, ground_truth)
    print(f"\n{key.capitalize()} Answer Evaluation Scores:", retriever_scores)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Pinecone Answer Evaluation Scores: {'context_utilization': 0.8865, 'faithfulness': 1.0000, 'answer_relevancy': 0.9545, 'context_recall': 1.0000}


In [3]:
# Usage Example
question = "What were the forecasts for U.S. crude oil production in the fourth quarter of 2016?"
aggregated_answer, all_responses, context, sources = get_aggregated_answer_and_sources(question)

# Format the output in a readable way
formatted_context = format_context(context)

output = f"""
Final Aggregated Answer: {aggregated_answer}

Intermediate Answers:
Chroma: {all_responses.get('chroma', 'No valid answer found')}
FAISS: {all_responses.get('faiss', 'No valid answer found')}
Pinecone: {all_responses.get('pinecone', 'No valid answer found')}

Context used for answer:
{formatted_context}
"""

print(output)

# Define the correct ground truth for the question
ground_truth = "The forecasts for U.S. crude oil production in the fourth quarter of 2016 were as follows: Current Forecast (as of December 6, 2016): 8.94 million barrels per day. Previous Forecast (as of November 8, 2016): 8.87 million barrels per day. This represents a percent change increase of 0.8% from the previous forecast to the current forecast for Q4 of 2016."

# Evaluate the final aggregated answer using the updated ground truth
final_scores = evaluate_answer(aggregated_answer, question, context, ground_truth)
print("\nFinal Aggregated Answer Evaluation Scores:", final_scores)

# Evaluate individual retriever answers using the updated ground truth
for key, answer in all_responses.items():
    retriever_scores = evaluate_answer(answer, question, context, ground_truth)
    print(f"\n{key.capitalize()} Answer Evaluation Scores:", retriever_scores)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Pinecone Answer Evaluation Scores: {'context_utilization': 0.7143, 'faithfulness': 1.0000, 'answer_relevancy': 0.9903, 'context_recall': 1.0000}


In [4]:
# Usage Example
question = "How did the number of Chapter 11 filings in Connecticut change between 1994 and 2004?"
aggregated_answer, all_responses, context, sources = get_aggregated_answer_and_sources(question)

# Format the output in a readable way
formatted_context = format_context(context)

output = f"""
Final Aggregated Answer: {aggregated_answer}

Intermediate Answers:
Chroma: {all_responses.get('chroma', 'No valid answer found')}
FAISS: {all_responses.get('faiss', 'No valid answer found')}
Pinecone: {all_responses.get('pinecone', 'No valid answer found')}

Context used for answer:
{formatted_context}
"""

print(output)

# Define the correct ground truth for the question
ground_truth = "Between 1994 and 2004, the number of Chapter 11 filings in Connecticut decreased. In 1994, there were 216 filings, and by 2004 the number had declined to 84 filings. This represents a significant reduction over the ten-year period."

# Evaluate the final aggregated answer using the updated ground truth
final_scores = evaluate_answer(aggregated_answer, question, context, ground_truth)
print("\nFinal Aggregated Answer Evaluation Scores:", final_scores)

# Evaluate individual retriever answers using the updated ground truth
for key, answer in all_responses.items():
    retriever_scores = evaluate_answer(answer, question, context, ground_truth)
    print(f"\n{key.capitalize()} Answer Evaluation Scores:", retriever_scores)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Pinecone Answer Evaluation Scores: {'context_utilization': 0.5278, 'faithfulness': 1.0000, 'answer_relevancy': 1.0000, 'context_recall': 1.0000}


In [5]:
# Usage Example
question = "What were the immediate actions taken by Nancy Powell and other volunteers following the Cosco Busan oil spill in San Francisco Bay?"
aggregated_answer, all_responses, context, sources = get_aggregated_answer_and_sources(question)

# Format the output in a readable way
formatted_context = format_context(context)

output = f"""
Final Aggregated Answer: {aggregated_answer}

Intermediate Answers:
Chroma: {all_responses.get('chroma', 'No valid answer found')}
FAISS: {all_responses.get('faiss', 'No valid answer found')}
Pinecone: {all_responses.get('pinecone', 'No valid answer found')}

Context used for answer:
{formatted_context}
"""

print(output)

# Define the summarized ground truth for the question
ground_truth = ("Following the Cosco Busan oil spill, Nancy Powell and other volunteers took immediate actions such as walking along the Bay to locate and protect oiled birds, purchasing equipment to rescue birds, organizing a bird-rescue network, and transporting birds to a wildlife center.")

# Evaluate the final aggregated answer using the updated ground truth
final_scores = evaluate_answer(aggregated_answer, question, context, ground_truth)
print("\nFinal Aggregated Answer Evaluation Scores:", final_scores)

# Evaluate individual retriever answers using the updated ground truth
for key, answer in all_responses.items():
    retriever_scores = evaluate_answer(answer, question, context, ground_truth)
    print(f"\n{key.capitalize()} Answer Evaluation Scores:", retriever_scores)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Pinecone Answer Evaluation Scores: {'context_utilization': 0.2784, 'faithfulness': 1.0000, 'answer_relevancy': 0.9582, 'context_recall': 1.0000}
